<table align="left">
  <tr>
    <td><img src="fleet.png" alt="fleet of icecream trucks" width="120"/></td>
    <td align="left"><h1>Lesson 4: Monitoring and Evaluating your Agent</h1></td>
  </tr>
</table>


<div style="background-color:#fff6ff; padding:13px; border-width:3px; border-color:#efe6ef; border-style:solid; border-radius:6px">
<p> 💻 &nbsp; <b>Access <code>requirements.txt</code> file:</b> 1) click on the <em>"File"</em> option on the top menu of the notebook and then 2) click on <em>"Open"</em>.

<p> ⬇ &nbsp; <b>Download Notebooks:</b> 1) click on the <em>"File"</em> option on the top menu of the notebook and then 2) click on <em>"Download as"</em> and select <em>"Notebook (.ipynb)"</em>.</p>

<p> 📒 &nbsp; For more help, please see the <em>"Appendix – Tips, Help, and Download"</em> Lesson.</p>

</div>

<p style="background-color:#f7fff8; padding:15px; border-width:3px; border-color:#e0f0e0; border-style:solid; border-radius:6px"> 🚨
&nbsp; <b>Different Run Results:</b> The output generated by AI chat models can vary with each execution due to their dynamic, probabilistic nature. Don't be surprised if your results differ from those shown in the video.</p>

## Setup Tracing

In [1]:
PROJECT_NAME = "Customer-Success"

In [2]:
import os
from dotenv import load_dotenv, find_dotenv
from phoenix.otel import register
from openinference.instrumentation.smolagents import SmolagentsInstrumentor

tracer_provider = register(
    project_name=PROJECT_NAME,
    #endpoint= get_phoenix_endpoint() + "v1/traces"
    endpoint = os.getenv('DLAI_LOCAL_URL').format(port='6006') + "v1/traces"
)
SmolagentsInstrumentor().instrument(tracer_provider=tracer_provider)

🔭 OpenTelemetry Tracing Details 🔭
|  Phoenix Project: Customer-Success
|  Span Processor: SimpleSpanProcessor
|  Collector Endpoint: https://s172-29-52-110p6006.lab-aws-production.deeplearning.ai/v1/traces
|  Transport: HTTP + protobuf
|  Transport Headers: {}
|  
|  Using a default SpanProcessor. `add_span_processor` will overwrite this default.
|  
|  ⚠️ WARNING: It is strongly advised to use a BatchSpanProcessor in production environments.
|  
|  `register` has set this TracerProvider as the global OpenTelemetry default.
|  To disable this behavior, call `register` with `set_global_tracer_provider=False`.



In [3]:
from dotenv import load_dotenv, find_dotenv
load_dotenv() # load variables from local .env file

from huggingface_hub import login

login(os.getenv('HF_API_KEY'))

In [4]:
from smolagents import HfApiModel

model=HfApiModel("Qwen/Qwen2.5-Coder-32B-Instruct", provider="together")

model([{"role": "user", "content": "Hello!"}])

ChatMessage(role=<MessageRole.ASSISTANT: 'assistant'>, content='Hello! How can I assist you today?', tool_calls=[], raw=ChatCompletionOutput(choices=[ChatCompletionOutputComplete(finish_reason='stop', index=0, message=ChatCompletionOutputMessage(role='assistant', content='Hello! How can I assist you today?', tool_call_id=None, tool_calls=[]), logprobs=None, seed=11276114009073930000)], created=1759243289, id='oDKJhnY-62bZhn-98747b3c19bb2996', model='Qwen/Qwen2.5-Coder-32B-Instruct', system_fingerprint=None, usage=ChatCompletionOutputUsage(completion_tokens=10, prompt_tokens=31, total_tokens=41, cached_tokens=0), object='chat.completion', prompt=[]))

In [5]:
# This is where you can access the display:
print(os.environ.get('DLAI_LOCAL_URL').format(port='6006'))

https://s172-29-52-110p6006.lab-aws-production.deeplearning.ai/


## Trace an agent run

In [6]:
from smolagents import HfApiModel, CodeAgent

agent = CodeAgent(model=model, tools=[])

>Note, the following line will sometimes get a timeout on the interface to the tracing package due to the networked interface. If this happens, try it again.


In [7]:
agent.run("What is the 100th Fibonacci number?")

╭──────────────────────────────────────────────────── New run ────────────────────────────────────────────────────╮
│                                                                                                                 │
│ What is the 100th Fibonacci number?                                                                             │
│                                                                                                                 │
╰─ HfApiModel - Qwen/Qwen2.5-Coder-32B-Instruct ──────────────────────────────────────────────────────────────────╯

━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ Step 1 ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

─ Executing parsed code: ──────────────────────────────────────────────────────────────────────────────────────── 
  def fibonacci(n):                                                                                                
      a, b = 0, 1                                                                                                  
      for _ in range(n):                                                                                           
          a, b = b, a + b                                                                                          
      return a                                                                                                     
                                                                                                                   
  fib_100 = fibonacci(100)                                                                                         
  final_answer(fib_100)                                                                                            
 ─────────────────────────────────────────────────────────────────────────────────────────────────────────────────

Out - Final answer: 354224848179261915075

[Step 1: Duration 0.14 seconds| Input tokens: 2,020 | Output tokens: 130]

354224848179261915075

In [8]:
# This is where you can access the display:
print(os.environ.get('DLAI_LOCAL_URL').format(port='6006'))

https://s172-29-52-110p6006.lab-aws-production.deeplearning.ai/


## Setup ice cream production system

In [9]:
from smolagents import tool
from typing import Dict

menu_prices = {"crepe nutella": 1.50, "vanilla ice cream": 2, "maple pancake": 1.}

ORDER_BOOK = {}

@tool
def place_order(quantities: Dict[str, int], session_id: int) -> None:
    """Places a pre-order of snacks.

    Args:
        quantities: a dictionary with names as keys and quantities as values
        session_id: the id for the client session
    """
    global ORDER_BOOK
    assert isinstance(quantities, dict), "Incorrect type for the input dictionary!"
    assert [key in menu_prices for key in quantities.keys()], f"All food names should be within {menu_prices.keys()}"
    ORDER_BOOK[session_id] = quantities

@tool
def get_prices(quantities: Dict[str, int]) -> str:
    """Gets price for certain quantities of ice cream.

    Args:
        quantities: a dictionary with names as keys and quantities as values
    """
    assert isinstance(quantities, dict), "Incorrect type for the input dictionary!"
    assert [key in menu_prices for key in quantities.keys()], f"All food names should be within {menu_prices.keys()}"
    total_price = sum([menu_prices[key] * value for key, value in quantities.items()])
    return (
        f"Given the current menu prices:\n{menu_prices}\nThe total price for your order would be: ${total_price}"
    )

In [10]:
order_agent = CodeAgent(
    tools=[place_order, get_prices],
    model=HfApiModel("Qwen/Qwen2.5-Coder-32B-Instruct", provider="together")
)

In [11]:
order_agent.run(
    "Could I come and collect one crepe nutella?",
    additional_args={"session_id": 192}
)

╭──────────────────────────────────────────────────── New run ────────────────────────────────────────────────────╮
│                                                                                                                 │
│ Could I come and collect one crepe nutella?                                                                     │
│ You have been provided with these additional arguments, that you can access using the keys as variables in your │
│ python code:                                                                                                    │
│ {'session_id': 192}.                                                                                            │
│                                                                                                                 │
╰─ HfApiModel - Qwen/Qwen2.5-Coder-32B-Instruct ──────────────────────────────────────────────────────────────────╯

━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ Step 1 ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

─ Executing parsed code: ──────────────────────────────────────────────────────────────────────────────────────── 
  place_order(quantities={'crepe nutella': 1}, session_id=session_id)                                              
  final_answer("Order placed for one crepe nutella.")                                                              
 ─────────────────────────────────────────────────────────────────────────────────────────────────────────────────

Out - Final answer: Order placed for one crepe nutella.

[Step 1: Duration 0.15 seconds| Input tokens: 2,193 | Output tokens: 68]

'Order placed for one crepe nutella.'

### Try multiple orders

In [12]:
client_requests = [
    ("Could I come and collect one crepe nutella?", "place_order"),
    ("What would be the price for 1 crêpe nutella + 2 pancakes?", "get_prices"),
    ("How did you start your ice-cream business?", None),
    ("What's the weather at the Louvre right now?", None),
    ("I'm not sure if I should order. I want a vanilla ice cream. but if it's more expensive than $1, I don't want it. If it's below, I'll order it, please.", "place_order")
]

In [13]:
for request in client_requests:
    order_agent.run(
        request[0],
        additional_args={"session_id": 0, "menu_prices": menu_prices}
    )

╭──────────────────────────────────────────────────── New run ────────────────────────────────────────────────────╮
│                                                                                                                 │
│ Could I come and collect one crepe nutella?                                                                     │
│ You have been provided with these additional arguments, that you can access using the keys as variables in your │
│ python code:                                                                                                    │
│ {'session_id': 0, 'menu_prices': {'crepe nutella': 1.5, 'vanilla ice cream': 2, 'maple pancake': 1.0}}.         │
│                                                                                                                 │
╰─ HfApiModel - Qwen/Qwen2.5-Coder-32B-Instruct ──────────────────────────────────────────────────────────────────╯

━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ Step 1 ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

─ Executing parsed code: ──────────────────────────────────────────────────────────────────────────────────────── 
  quantities = {'crepe nutella': 1}                                                                                
  place_order(quantities=quantities, session_id=session_id)                                                        
  final_answer("Order placed for one crepe nutella.")                                                              
 ─────────────────────────────────────────────────────────────────────────────────────────────────────────────────

Out - Final answer: Order placed for one crepe nutella.

[Step 1: Duration 0.14 seconds| Input tokens: 2,227 | Output tokens: 93]

...

[Step 1: Duration 0.08 seconds| Input tokens: 2,260 | Output tokens: 89]

━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ Step 2 ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

─ Executing parsed code: ──────────────────────────────────────────────────────────────────────────────────────── 
  final_answer("I will not order the vanilla ice cream.")                                                          
 ─────────────────────────────────────────────────────────────────────────────────────────────────────────────────

Out - Final answer: I will not order the vanilla ice cream.

[Step 2: Duration 0.11 seconds| Input tokens: 4,723 | Output tokens: 138]

In [14]:
import phoenix as px

spans = px.Client().get_spans_dataframe(project_name=PROJECT_NAME)
spans.head(20)

/usr/local/lib/python3.11/site-packages/phoenix/utilities/client.py:51: UserWarning: ⚠️⚠️ The Phoenix server (12.2.0) and client (8.19.1) versions are severely mismatched. Upgrade  either the client or server to ensure API compatibility ⚠️⚠️
  warnings.warn(


,name,span_kind,parent_id,start_time,end_time,status_code,status_message,events,context.span_id,context.trace_id,...,attributes.llm.token_count.completion,attributes.llm.token_count.prompt,attributes.llm.output_messages,attributes.output.value,attributes.llm.invocation_parameters,attributes.llm.token_count.total,attributes.llm.model_name,attributes.llm.input_messages,attributes.input.mime_type,attributes.smolagents
context.span_id,,,,,,,,,,,,,,,,,,,,,
8ef320f952b55ef2,FinalAnswerTool,TOOL,132e03c6c059226b,2025-10-04 14:35:21.844657+00:00,2025-10-04 14:35:21.844741+00:00,OK,,[],8ef320f952b55ef2,0a1ea5acf2471b37d4e74f674888a287,...,NaN,NaN,None,None,None,NaN,None,None,None,None
e61a134e71367db7,HfApiModel.__call__,LLM,132e03c6c059226b,2025-10-04 14:35:21.802404+00:00,2025-10-04 14:35:21.816422+00:00,OK,,[],e61a134e71367db7,0a1ea5acf2471b37d4e74f674888a287,...,49.0,2463.0,"[{'message.role': 'assistant', 'message.conten...","{""role"": ""assistant"", ""content"": ""Thought: The...",{},2512.0,Qwen/Qwen2.5-Coder-32B-Instruct,"[{'message.role': 'system', 'message.content':...",application/json,None
132e03c6c059226b,Step 2,CHAIN,854e10a717712c23,2025-10-04 14:35:21.802110+00:00,2025-10-04 14:35:21.885975+00:00,OK,,[],132e03c6c059226b,0a1ea5acf2471b37d4e74f674888a287,...,NaN,NaN,None,Execution logs:\nLast output from code snippet...,None,NaN,None,None,None,None
9c5c478e23cd2f5b,HfApiModel.__call__,LLM,b26551ddcf004c26,2025-10-04 14:35:21.721565+00:00,2025-10-04 14:35:21.735288+00:00,OK,,[],9c5c478e23cd2f5b,0a1ea5acf2471b37d4e74f674888a287,...,89.0,2260.0,"[{'message.role': 'assistant', 'message.conten...","{""role"": ""assistant"", ""content"": ""Thought: I n...",{},2349.0,Qwen/Qwen2.5-Coder-32B-Instruct,"[{'message.role': 'system', 'message.content':...",application/json,None
b26551ddcf004c26,Step 1,CHAIN,854e10a717712c23,2025-10-04 14:35:21.721367+00:00,2025-10-04 14:35:21.768527+00:00,OK,,[],b26551ddcf004c26,0a1ea5acf2471b37d4e74f674888a287,...,NaN,NaN,None,Execution logs:\nThe price of vanilla ice crea...,None,NaN,None,None,None,None
2675b5092e974a23,FinalAnswerTool,TOOL,1779e9775c82cf3f,2025-10-04 14:35:21.638643+00:00,2025-10-04 14:35:21.638697+00:00,OK,,[],2675b5092e974a23,12a2e1e0f461ec9d624db0125f1654a5,...,NaN,NaN,None,None,None,NaN,None,None,None,None
5d30204d91ecb73c,HfApiModel.__call__,LLM,1779e9775c82cf3f,2025-10-04 14:35:21.598295+00:00,2025-10-04 14:35:21.613216+00:00,OK,,[],5d30204d91ecb73c,12a2e1e0f461ec9d624db0125f1654a5,...,83.0,2696.0,"[{'message.role': 'assistant', 'message.conten...","{""role"": ""assistant"", ""content"": ""Thought: The...",{},2779.0,Qwen/Qwen2.5-Coder-32B-Instruct,"[{'message.role': 'system', 'message.content':...",application/json,None
1779e9775c82cf3f,Step 3,CHAIN,52775836e0f289a4,2025-10-04 14:35:21.598024+00:00,2025-10-04 14:35:21.669148+00:00,OK,,[],1779e9775c82cf3f,12a2e1e0f461ec9d624db0125f1654a5,...,NaN,NaN,None,Execution logs:\nLast output from code snippet...,None,NaN,None,None,None,None
563f510262bc791e,HfApiModel.__call__,LLM,83c57ce3dfc837b6,2025-10-04 14:35:21.526242+00:00,2025-10-04 14:35:21.541717+00:00,OK,,[],563f510262bc791e,12a2e1e0f461ec9d624db0125f1654a5,...,123.0,2446.0,"[{'message.role': 'assistant', 'message.conten...","{""role"": ""assistant"", ""content"": ""Thought: Sin...",{},2569.0,Qwen/Qwen2.5-Coder-32B-Instruct,"[{'message.role': 'system', 'message.content':...",application/json,None


### Add processing to extract desired information

In [15]:
import pandas as pd
import json

agents = spans[spans['span_kind'] == 'AGENT'].copy()
agents['task'] = agents['attributes.input.value'].apply(
    lambda x: json.loads(x).get('task') if isinstance(x, str) else None
)

tools = spans.loc[
    spans['span_kind'] == 'TOOL',
    ["attributes.tool.name", "attributes.input.value", "context.trace_id"]
].copy()

tools_per_task = agents[
    ["name", "start_time", "task", "context.trace_id"]
].merge(
    tools,
    on="context.trace_id",
    how="left",
)
tools_per_task.head()

,name,start_time,task,context.trace_id,attributes.tool.name,attributes.input.value
0,CodeAgent.run,2025-10-04 14:35:21.434890+00:00,What's the weather at the Louvre right now?,12a2e1e0f461ec9d624db0125f1654a5,final_answer,"{""args"": [""Sunny, 22°C, 75% humidity, 10 km/h ..."
1,CodeAgent.run,2025-10-04 14:35:21.176414+00:00,How did you start your ice-cream business?,b4a8702db84a4c28fa3f469c210abfe9,final_answer,"{""args"": [""I started my ice-cream business wit..."
2,CodeAgent.run,2025-10-04 14:35:21.176414+00:00,How did you start your ice-cream business?,b4a8702db84a4c28fa3f469c210abfe9,get_prices,"{""args"": [], ""sanitize_inputs_outputs"": false,..."
3,CodeAgent.run,2025-10-04 14:35:20.920910+00:00,What would be the price for 1 crêpe nutella + ...,63e8d7c2a84f05f8b0e2fb1939af00df,final_answer,"{""args"": [3.5], ""sanitize_inputs_outputs"": fal..."
4,CodeAgent.run,2025-10-04 14:35:20.920910+00:00,What would be the price for 1 crêpe nutella + ...,63e8d7c2a84f05f8b0e2fb1939af00df,get_prices,"{""args"": [], ""sanitize_inputs_outputs"": false,..."


### Now, compare tool calls with exected tool calls

In [16]:
def score_request(expected_tool: str, tool_calls: list):
    if expected_tool is None:
        return tool_calls == set(["final_answer"])
    else:
        return expected_tool in tool_calls

results = []
for request, expected_tool in client_requests:
    tool_calls = set(tools_per_task.loc[tools_per_task["task"] == request, "attributes.tool.name"].tolist())
    results.append(
        {
            "request": request,
            "tool_calls_performed": tool_calls,
            "is_correct": score_request(expected_tool, tool_calls)
        }
    )
pd.DataFrame(results)

,request,tool_calls_performed,is_correct
0,Could I come and collect one crepe nutella?,"{final_answer, place_order}",True
1,What would be the price for 1 crêpe nutella + ...,"{final_answer, get_prices}",True
2,How did you start your ice-cream business?,"{final_answer, get_prices}",False
3,What's the weather at the Louvre right now?,{final_answer},True
4,I'm not sure if I should order. I want a vanil...,{},False
